# public, private, protected property

- python은 **public**만 존재한다.
- 밑줄을 통한 네이밍으로 **public**, **private**, **protected**로 구분하고, 그에 맞게 쓰일 것을 **기대**한다.
- 밑줄로 시작하는 속성은 해당 객체에 대해 **private**을 의미한다.

```python
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60
```

```bash
>>> conn = Connector("postgresql://localhost")
>>> conn.soure
'postgresql://localhost'
>>> conn._timeout
60
>>> conn.__dict__
{'source':'postgresql://localhost', '_timeout': 60}
```

- 밑줄이 쳐저 있더라도, 접근이 가능하다.
- 개발자가, 이를 인식하여 private으로 사용되기로 기대될 변수명 표기가 있다면, 이를 맞춰서 사용하는게 좋다.

# misunderstanding :: name mangling

```python
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60
        
    def connect(self):
        print('connecting with {0}s'.format(self.__timeout))
        # ...
```

```bash
>>> conn = Connector("postgresql://localhost")
>>> conn.connect()
'connecting with 60s'
>>> conn.__timeout
Trackback (most recent call last):
    File "<stdin>", line 1, in <module>
    AttributeError : 'Connector' object has no attribute '__timeout'
```

- 이는 *AttributeError*이지, **접근 불가**이거나, **private**이라는 이야기는 아니다.
- 밑줄을 두 개를 사용하면 실제로 파이썬은 다른 이름을 만든다:: name mangling
    - "_<class-name>__<attribute-name>"인 경우 `_Connector__timeout`이라는 속성을 만든다.
    
```bash
>>> varss(conn)
{'source': 'postgresql://localhost', '_Connector__timeout': 60}
>>> conn._Connector__timeout
60
>>> conn._Connector__timeout = 30
>>> conn.connect()
connecting with 30s
```

- 두번 이상의 밑줄은 **여러 번 확장되는 클래스의 메서드 이름을 충돌 없이 오버라이드하기 위해서 만들어졌다.**
- 이중 밑줄은 파이썬스러운 코드가 아니다. 속성을 **private**으로 정의하는 경우 하나의 밑줄을 사용하고, 파이썬스러운 관습을 지키자.

## Property

- 객체에 값을 저장해야 할 경우 일반적인 속성(attribute)을 사용할 수 있다.
- 때로는 객체의 상태나 다른 속성의 값을 기반으로 어떤 계산을 하려고 할 때도 있다.
    
- 자바와 같은 언어에서는 접근 메소드(getter & setter)를 만든다.
- python에서는 프로퍼티를 사용한다.

```python
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None
        
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
            
        self._email = new_email
```

- `@property` 메소드는 private 속성인 email 값을 반환한다.
- `@email.setter`는 <user>.email = <new_email>이 실행될 때, 호출되는 코드다.
    
```bash
>>> u1 = User("jsmith")
>>> u1.email = "jsmith@"
Trackback (most recent call last):
...
유효한 이메일이 아니므로 jsmith@ 값을 사용할 수 없음
>>> u1.email = "jsmith@g.co"
>>> u1.email
'jsmith@g.co'
```

- 이러한 프로퍼티는 명령-쿼리 분리 원칙(command and query separation - CC08)을 따르기 위한 좋은 방법이다.